In [2]:
import tensorflow as tf
import pandas as pd
import platform
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
subm = pd.read_csv('./data/sample_submission.csv')

In [4]:
train.head() 

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
test.head(5)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [6]:
lens = train.comment_text.str.len()
lens2 =test.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

(394.0732213246768, 590.7202819048919, 5000)

In [7]:
hist = lens.hist(bins=100)

hist.set_xlim(0, 1000)
start, end = hist.get_xlim()
hist.xaxis.set_ticks(np.arange(start, end, 100));

In [8]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805,0.898321
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420,0.302226
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
COMMENT = 'comment_text'
train[COMMENT].fillna('unknown', inplace=True)
test[COMMENT].fillna('unknown', inplace=True)

TfidVectorizer creates two sparse matrices (train_docs, test_docs) that represent the tfids of each of the words inside of a document.

In [11]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [12]:
vectorizer = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2),
                             max_df=0.9, min_df=3, strip_accents='unicode', sublinear_tf=1)

train_docs = vectorizer.fit_transform(train.comment_text)

test_docs = vectorizer.transform(test.comment_text)

In [13]:
x = train_docs

Next is a fitting to a logit model. The solver 'saga' was used because the default 'liblinear' would fail with the error "'liblinear' does not support multinomial classification". We are inputing into the fitting model only the column that corresponds to 'toxic'. This model basically classifies comments into toxic/non-toxic classes. 

In [14]:
def prob(y, y_i):
    p = x[y==y_i].sum(0)
    return (p + 1) / ((y==y_i).sum() + 1)

In [15]:
def get_model(y):
    y = y.values
    r = np.log(prob(y, 1) / prob(y, 0))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r
    

In [16]:
preds = np.zeros((len(test), len(label_cols)))

for (i, j) in enumerate(label_cols):
    print('fit ', j)
    m,r = get_model(train[j])
    preds[:,i] = m.predict_proba(test_docs.multiply(r))[:,1]

    

fit  toxic
fit  severe_toxic
fit  obscene
fit  threat
fit  insult
fit  identity_hate


In [17]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)

In [18]:
type(subm["id"])

pandas.core.series.Series

Some slicing needs to be done, hopefully random, to construct an actual train/validation set. The original test file does not have target values and cannot be used for validation. Next is a fitting to a logit model. The solver 'saga' was used because the default 'liblinear' would fail with the error "'liblinear' does not support multinomial classification". We are inputing into the fitting model only the column that corresponds to 'toxic'. This model basically classifies comments into toxic/non-toxic classes. 

In [19]:
log_reg = LogisticRegression(multi_class='multinomial', penalty ='l1',solver='saga', tol=0.1)
model = log_reg.fit(count_vectorized_data[:int(target.shape[0]*0.6),], target[:int(target.shape[0]*0.6),0])
score = log_reg.score(count_vectorized_data[int(target.shape[0]*0.6)+1:,], target[int(target.shape[0]*0.6)+1:,0])
print("The score for this class is: %.3f" %score) 

NameError: name 'count_vectorized_data' is not defined

In [ ]:
train_samples  = int(target.shape[0]*0.6) 

Fitting the model for every class. Not necesarily very smart. This s

In [ ]:
scores =[] 
for i in range(target.shape[1]):
    model = log_reg.fit(count_vectorized_data[:train_samples,], target[:train_samples,i])
    score = log_reg.score(count_vectorized_data[train_samples+1:,], target[train_samples+1:,0])
    print("The score for this class is: %.3f" %score)
    scores.append(score)
    
scores